In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, LayerNormalization, MultiHeadAttention, Dropout, Conv1D
from tensorflow.keras.layers import GlobalAveragePooling1D, Concatenate, Add, Flatten, Embedding, Bidirectional, GRU
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1_l2
import os
import time
from datetime import datetime

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ScienceProjects/WaterLevelPredict/data/mucnuoc_gio_preprocess.csv')
df.head(3)

,date,q120,q55,q64,q66,q69
0,2014-01-01 01:00:00,-0.94,-8.00,-4.58,-1.45,-9.01
1,2014-01-01 03:00:00,-0.94,-7.98,-4.57,-1.45,-9.00
2,2014-01-01 05:00:00,-0.94,-7.95,-4.58,-1.45,-9.00


In [ ]:
# --- BƯỚC 1: ĐỌC DỮ LIỆU ---
# Thay đổi đường dẫn file nếu cần
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date')

# Các cột mực nước cần dùng
features = ['q64']
target = 'q64'  # Cột mục tiêu

In [ ]:
# --- BƯỚC 2: CHUẨN HÓA DỮ LIỆU ---
# Sử dụng MinMaxScaler cho các tính năng chính
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df[features])
scaled_df = pd.DataFrame(scaled_data, columns=features)

# Tạo thêm các tính năng mới từ dữ liệu mực nước
for feature in features:
    # Thêm rolling statistics (trung bình và độ lệch chuẩn của 24 mẫu)
    scaled_df[f'{feature}_rolling_mean_24'] = scaled_df[feature].rolling(window=24).mean().fillna(method='bfill')
    scaled_df[f'{feature}_rolling_std_24'] = scaled_df[feature].rolling(window=24).std().fillna(method='bfill')

    # Thêm tốc độ thay đổi (rate of change)
    scaled_df[f'{feature}_diff'] = scaled_df[feature].diff().fillna(0)

# Thêm tính năng từ thời gian - sẽ thêm trực tiếp vào scaled_df
# Trích xuất thông tin thời gian từ cột 'date'
scaled_df['month'] = df['date'].dt.month / 12.0  # Chuẩn hóa về 0-1
scaled_df['day_of_month'] = df['date'].dt.day / 31.0  # Chuẩn hóa về 0-1
scaled_df['day_of_week'] = df['date'].dt.dayofweek / 6.0  # Chuẩn hóa về 0-1
scaled_df['hour'] = df['date'].dt.hour / 23.0  # Chuẩn hóa về 0-1

# Thêm biến chu kỳ cho tháng, tuần, ngày (tính toán theo hàm sin và cos)
scaled_df['month_sin'] = np.sin(2 * np.pi * df['date'].dt.month / 12)
scaled_df['month_cos'] = np.cos(2 * np.pi * df['date'].dt.month / 12)
scaled_df['day_sin'] = np.sin(2 * np.pi * df['date'].dt.day / 31)
scaled_df['day_cos'] = np.cos(2 * np.pi * df['date'].dt.day / 31)

# Danh sách tất cả các tính năng sau khi mở rộng
extended_features = scaled_df.columns.tolist()
print(f"Số lượng tính năng đã được mở rộng: {len(extended_features)}")
print(f"Các tính năng: {extended_features}")

Số lượng tính năng đã được mở rộng: 12
Các tính năng: ['q64', 'q64_rolling_mean_24', 'q64_rolling_std_24', 'q64_diff', 'month', 'day_of_month', 'day_of_week', 'hour', 'month_sin', 'month_cos', 'day_sin', 'day_cos']


/tmp/ipython-input-5-2700539258.py:10: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  scaled_df[f'{feature}_rolling_mean_24'] = scaled_df[feature].rolling(window=24).mean().fillna(method='bfill')
/tmp/ipython-input-5-2700539258.py:11: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  scaled_df[f'{feature}_rolling_std_24'] = scaled_df[feature].rolling(window=24).std().fillna(method='bfill')


In [ ]:
# --- BƯỚC 3: TẠO DỮ LIỆU WINDOW - PHIÊN BẢN SỬA LỖI ---
past_window = 24   # 30 ngày trước
future_window = 72 # Dự báo SEQUENCE 30 ngày sau (không phải mean!)

# Sử dụng tất cả các tính năng mở rộng
all_scaled_data = scaled_df[extended_features].values.astype(np.float32)

X, y = [], []
target_idx = extended_features.index(target)

for i in range(len(all_scaled_data) - past_window - future_window):
    # Dữ liệu đầu vào: cửa sổ quá khứ với tất cả tính năng
    X_window = all_scaled_data[i:i+past_window]

    # ✅ FIXED: Mục tiêu là SEQUENCE future_window timesteps, KHÔNG PHẢI MEAN!
    y_future_sequence = all_scaled_data[i+past_window:i+past_window+future_window, target_idx]

    X.append(X_window)
    y.append(y_future_sequence)

X = np.array(X, dtype=np.float32)  # Shape: (samples, past_window, n_features)
y = np.array(y, dtype=np.float32)  # Shape: (samples, future_window) - KHÔNG reshape thành (-1, 1)

print(f"✅ Tạo tập dữ liệu với {len(X)} mẫu")
print(f"📊 X shape: {X.shape} (samples, past_window, features)")
print(f"📊 y shape: {y.shape} (samples, future_window)")
print(f"🎯 Target: Dự đoán {future_window} timesteps sequence thay vì 1 giá trị mean")



✅ Tạo tập dữ liệu với 47407 mẫu
📊 X shape: (47407, 24, 12) (samples, past_window, features)
📊 y shape: (47407, 72) (samples, future_window)
🎯 Target: Dự đoán 72 timesteps sequence thay vì 1 giá trị mean


In [ ]:
# --- BƯỚC 4: CHIA TẬP TRAIN/TEST ---
split_idx = int(len(X) * 0.8)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

print(f"Train set: {X_train.shape}, {y_train.shape}")
print(f"Test set: {X_test.shape}, {y_test.shape}")


Train set: (37925, 24, 12), (37925, 72)
Test set: (9482, 24, 12), (9482, 72)


In [ ]:
# --- BƯỚC 5: XÂY DỰNG MÔ HÌNH TRANSFORMER - FIXED VERSION ---
def positional_encoding(length, depth):
    """
    Tạo mã hóa vị trí (positional encoding) cho Transformer
    """
    depth = depth/2
    positions = np.arange(length)[:, np.newaxis]
    depths = np.arange(depth)[np.newaxis, :]/depth
    angle_rates = 1 / (10000**depths)
    angle_rads = positions * angle_rates

    pos_encoding = np.concatenate([np.sin(angle_rads), np.cos(angle_rads)], axis=-1)
    return tf.cast(pos_encoding, dtype=tf.float32)

def transformer_encoder_block(inputs, head_size, num_heads, ff_dim, dropout=0.1, activation='gelu'):
    """
    Khối transformer encoder cải tiến
    """
    # Pre-normalization cho attention
    x = LayerNormalization(epsilon=1e-6)(inputs)

    # Multi-head attention
    attn_output = MultiHeadAttention(
        key_dim=head_size,
        num_heads=num_heads,
        dropout=dropout
    )(x, x)

    # Residual connection
    attn_output = Dropout(dropout)(attn_output)
    x1 = Add()([inputs, attn_output])

    # Pre-normalization cho feed-forward
    x2 = LayerNormalization(epsilon=1e-6)(x1)

    # Feed-forward network
    ffn_output = Dense(ff_dim, activation=activation)(x2)
    ffn_output = Dropout(dropout)(ffn_output)
    ffn_output = Dense(inputs.shape[-1])(ffn_output)
    ffn_output = Dropout(dropout)(ffn_output)

    # Residual connection 2
    return Add()([x1, ffn_output])

def build_transformer_model_fixed(input_shape, future_window, head_size=64, num_heads=8, ff_dim=256,
                                 num_transformer_blocks=4, mlp_units=[128, 64],
                                 dropout=0.1, mlp_dropout=0.1, learning_rate=1e-4,
                                 use_positional_encoding=True, activation='gelu'):
    """
    Xây dựng mô hình Transformer cho dự báo mực nước - PHIÊN BẢN SỬA LỖI
    """
    inputs = Input(shape=input_shape)

    # Thêm positional encoding nếu được chọn
    if use_positional_encoding:
        x = inputs + positional_encoding(input_shape[0], input_shape[1])
    else:
        x = inputs

    # Khối Transformer
    for _ in range(num_transformer_blocks):
        x = transformer_encoder_block(
            x,
            head_size=head_size,
            num_heads=num_heads,
            ff_dim=ff_dim,
            dropout=dropout,
            activation=activation
        )

    # Global pooling
    x = GlobalAveragePooling1D()(x)

    # MLP layers
    for dim in mlp_units:
        x = Dense(dim, activation=activation)(x)
        x = LayerNormalization(epsilon=1e-6)(x)
        x = Dropout(mlp_dropout)(x)

    # ✅ FIXED: Output = future_window thay vì 1
    outputs = Dense(future_window)(x)

    model = Model(inputs, outputs)

    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='mse',
        metrics=['mae']
    )

    return model

In [ ]:
# --- BƯỚC 6: HUẤN LUYỆN MÔ HÌNH ---
# Tạo thư mục lưu trữ mô hình
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
model_dir = f"water_level_model_{timestamp}"
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# Xây dựng mô hình với các tham số tối ưu cho A100
input_shape = (past_window, len(extended_features))
model = build_transformer_model_fixed(  # ✅ FIXED: Thêm future_window parameter
    input_shape=input_shape,
    future_window=future_window,  # ✅ THÊM PARAMETER QUAN TRỌNG
    head_size=64,
    num_heads=16,  # Tăng số heads để nắm bắt nhiều mối quan hệ hơn
    ff_dim=512,    # Tăng kích thước feed-forward network
    num_transformer_blocks=6,  # Tăng số lượng transformer blocks
    mlp_units=[256, 128, 64],  # Mạng MLP sâu hơn
    dropout=0.2,   # Tăng dropout để giảm overfitting
    mlp_dropout=0.2,
    learning_rate=5e-5,  # Learning rate thấp hơn để ổn định
    use_positional_encoding=True,
    activation='gelu'  # Sử dụng GELU thay vì ReLU
)

model.summary()

# Callbacks
callbacks = [
    ModelCheckpoint(
        filepath=os.path.join(model_dir, "best_model.h5"),
        save_best_only=True,
        monitor='val_loss',
        verbose=1
    ),
    EarlyStopping(
        patience=3,  # ✅ Tăng patience cho sequence-to-sequence
        restore_best_weights=True,
        monitor='val_loss',
        verbose=1
    ),
    ReduceLROnPlateau(
        factor=0.5,
        patience=12,  # ✅ Tăng patience
        min_lr=1e-7,
        monitor='val_loss',
        verbose=1
    ),
    TensorBoard(
        log_dir=os.path.join(model_dir, "logs"),
        histogram_freq=1,
        write_graph=True
    )
]

# Huấn luyện mô hình
print("🚀 Bắt đầu huấn luyện mô hình Transformer...")
start_time = time.time()

history = model.fit(
    X_train, y_train,
    epochs=30,  # ✅ Tăng epochs cho sequence-to-sequence
    batch_size=32,  # ✅ Giảm batch size vì output lớn hơn
    validation_data=(X_test, y_test),
    callbacks=callbacks,
    verbose=1
)

training_time = time.time() - start_time
print(f"⏱️ Thời gian huấn luyện: {training_time:.2f} giây")


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 24, 12)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_65 (Add)        │ (None, 24, 12)    │          0 │ input_layer_5[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 24, 12)    │         24 │ add_65[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 24, 12)    │     52,236 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_136         │ (None, 24, 12)    │          0 │ multi_head_atten… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_66 (Add)        │ (None, 24, 12)    │          0 │ add_65[0][0],     │
│                     │                   │            │ dropout_136[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 24, 12)    │         24 │ add_66[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_80 (Dense)    │ (None, 24, 512)   │      6,656 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_137         │ (None, 24, 512)   │          0 │ dense_80[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_81 (Dense)    │ (None, 24, 12)    │      6,156 │ dropout_137[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_138         │ (None, 24, 12)    │          0 │ dense_81[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_67 (Add)        │ (None, 24, 12)    │          0 │ add_66[0][0],     │
│                     │                   │            │ dropout_138[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 24, 12)    │         24 │ add_67[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 24, 12)    │     52,236 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_140         │ (None, 24, 12)    │          0 │ multi_head_atten… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_68 (Add)        │ (None, 24, 12)    │          0 │ add_67[0][0],     │
│                     │                   │            │ dropout_140[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 24, 12)    │         24 │ add_68[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_82 (Dense)    │ (None, 24, 512)   │      6,656 │ layer_normalizat

 Total params: 440,632 (1.68 MB)

 Trainable params: 440,632 (1.68 MB)

 Non-trainable params: 0 (0.00 B)

...

1186/1186 ━━━━━━━━━━━━━━━━━━━━ 13s 11ms/step - loss: 0.1221 - mae: 0.2718 - val_loss: 0.0257 - val_mae: 0.1248 - learning_rate: 5.0000e-05
Epoch 3/30
1184/1186 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0369 - mae: 0.1468
Epoch 3: val_loss improved from 0.02568 to 0.00740, saving model to water_level_model_20250620-151437/best_model.h5


1186/1186 ━━━━━━━━━━━━━━━━━━━━ 13s 11ms/step - loss: 0.0368 - mae: 0.1468 - val_loss: 0.0074 - val_mae: 0.0649 - learning_rate: 5.0000e-05
Epoch 4/30
1185/1186 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0166 - mae: 0.0979
Epoch 4: val_loss improved from 0.00740 to 0.00287, saving model to water_level_model_20250620-151437/best_model.h5


1186/1186 ━━━━━━━━━━━━━━━━━━━━ 13s 11ms/step - loss: 0.0166 - mae: 0.0979 - val_loss: 0.0029 - val_mae: 0.0402 - learning_rate: 5.0000e-05
Epoch 5/30
1186/1186 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0105 - mae: 0.0778
Epoch 5: val_loss improved from 0.00287 to 0.00185, saving model to water_level_model_20250620-151437/best_model.h5


1186/1186 ━━━━━━━━━━━━━━━━━━━━ 13s 11ms/step - loss: 0.0105 - mae: 0.0778 - val_loss: 0.0019 - val_mae: 0.0336 - learning_rate: 5.0000e-05
Epoch 6/30
1185/1186 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0073 - mae: 0.0646
Epoch 6: val_loss improved from 0.00185 to 0.00081, saving model to water_level_model_20250620-151437/best_model.h5


1186/1186 ━━━━━━━━━━━━━━━━━━━━ 13s 11ms/step - loss: 0.0073 - mae: 0.0646 - val_loss: 8.1088e-04 - val_mae: 0.0216 - learning_rate: 5.0000e-05
Epoch 7/30
1186/1186 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0053 - mae: 0.0549
Epoch 7: val_loss did not improve from 0.00081
1186/1186 ━━━━━━━━━━━━━━━━━━━━ 13s 11ms/step - loss: 0.0053 - mae: 0.0549 - val_loss: 0.0019 - val_mae: 0.0352 - learning_rate: 5.0000e-05
Epoch 8/30
1184/1186 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0040 - mae: 0.0476
Epoch 8: val_loss did not improve from 0.00081
1186/1186 ━━━━━━━━━━━━━━━━━━━━ 13s 11ms/step - loss: 0.0040 - mae: 0.0476 - val_loss: 8.1652e-04 - val_mae: 0.0218 - learning_rate: 5.0000e-05
Epoch 9/30
1186/1186 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0031 - mae: 0.0422
Epoch 9: val_loss did not improve from 0.00081
1186/1186 ━━━━━━━━━━━━━━━━━━━━ 13s 11ms/step - loss: 0.0031 - mae: 0.0422 - val_loss: 0.0016 - val_mae: 0.0321 - learning_rate: 5.0000e-05
Epoch 9: early stopping
Restoring model weight

In [ ]:

# --- BƯỚC 7: ĐÁNH GIÁ MÔ HÌNH - FIXED VERSION ---
# Dự đoán với tập test
print("🔍 Đang đánh giá mô hình...")
y_pred = model.predict(X_test)  # Shape: (test_samples, future_window)

# Chuyển ngược giá trị dự đoán về đơn vị ban đầu
q64_index = features.index('q64')
q64_min = scaler.data_min_[q64_index]
q64_max = scaler.data_max_[q64_index]

y_pred_inv = y_pred * (q64_max - q64_min) + q64_min
y_test_inv = y_test * (q64_max - q64_min) + q64_min

# ✅ FIXED: Tính các chỉ số đánh giá cho sequence prediction
# Overall metrics (flatten all predictions)
mae_overall = mean_absolute_error(y_test_inv.flatten(), y_pred_inv.flatten())
mse_overall = mean_squared_error(y_test_inv.flatten(), y_pred_inv.flatten())
rmse_overall = np.sqrt(mse_overall)
r2_overall = r2_score(y_test_inv.flatten(), y_pred_inv.flatten())

print("\n=== ĐÁNH GIÁ MÔ HÌNH TRANSFORMER - FIXED VERSION ===")
print(f"📊 Overall Performance:")
print(f"   MAE: {mae_overall:.4f}")
print(f"   MSE: {mse_overall:.4f}")
print(f"   RMSE: {rmse_overall:.4f}")
print(f"   R²: {r2_overall:.4f}")
print(f"   Accuracy: {r2_overall*100:.2f}%")

# ✅ THÊM: Per-timestep analysis
print(f"\n🔍 Per-timestep Performance (first 10 steps):")
for t in range(min(10, future_window)):
    mae_t = mean_absolute_error(y_test_inv[:, t], y_pred_inv[:, t])
    r2_t = r2_score(y_test_inv[:, t], y_pred_inv[:, t])
    print(f"   Step {t+1:2d}: MAE={mae_t:.4f}, R²={r2_t:.4f}")

# ✅ THÊM: Horizon analysis
short_horizon = min(30, future_window // 4)
long_horizon_start = max(future_window - 30, future_window * 3 // 4)

mae_short = mean_absolute_error(
    y_test_inv[:, :short_horizon].flatten(),
    y_pred_inv[:, :short_horizon].flatten()
)
mae_long = mean_absolute_error(
    y_test_inv[:, long_horizon_start:].flatten(),
    y_pred_inv[:, long_horizon_start:].flatten()
)

r2_short = r2_score(
    y_test_inv[:, :short_horizon].flatten(),
    y_pred_inv[:, :short_horizon].flatten()
)
r2_long = r2_score(
    y_test_inv[:, long_horizon_start:].flatten(),
    y_pred_inv[:, long_horizon_start:].flatten()
)

print(f"\n📈 Horizon Analysis:")
print(f"   Short-term (1-{short_horizon}): MAE={mae_short:.4f}, R²={r2_short:.4f}")
print(f"   Long-term ({long_horizon_start+1}-{future_window}): MAE={mae_long:.4f}, R²={r2_long:.4f}")

# ✅ THÊM: Lưu kết quả để so sánh
transformer_results = {
    'model_type': 'Transformer',
    'past_window': past_window,
    'future_window': future_window,
    'y_test': y_test_inv,
    'y_pred': y_pred_inv,
    'mae': mae_overall,
    'mse': mse_overall,
    'rmse': rmse_overall,
    'r2': r2_overall,
    'mae_short': mae_short,
    'mae_long': mae_long,
    'r2_short': r2_short,
    'r2_long': r2_long,
    'history': history,
    'epochs_trained': len(history.history['loss']),
    'training_time': training_time,
    'model': model
}

print(f"\n🎯 Transformer training completed!")
print(f"   Best performance: R² = {r2_overall:.4f}")
print(f"   Training time: {training_time:.2f}s")
print(f"   Epochs trained: {len(history.history['loss'])}")
print(f"   Model saved to: {model_dir}")

🔍 Đang đánh giá mô hình...
297/297 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step

=== ĐÁNH GIÁ MÔ HÌNH TRANSFORMER - FIXED VERSION ===
📊 Overall Performance:
   MAE: 0.0862
   MSE: 0.0130
   RMSE: 0.1139
   R²: 0.9526
   Accuracy: 95.26%

🔍 Per-timestep Performance (first 10 steps):
   Step  1: MAE=0.0772, R²=0.9640
   Step  2: MAE=0.0554, R²=0.9815
   Step  3: MAE=0.0619, R²=0.9775
   Step  4: MAE=0.0680, R²=0.9719
   Step  5: MAE=0.0614, R²=0.9761
   Step  6: MAE=0.0617, R²=0.9758
   Step  7: MAE=0.0684, R²=0.9730
   Step  8: MAE=0.2008, R²=0.8072
   Step  9: MAE=0.0646, R²=0.9748
   Step 10: MAE=0.0632, R²=0.9741

📈 Horizon Analysis:
   Short-term (1-18): MAE=0.0756, R²=0.9621
   Long-term (55-72): MAE=0.0963, R²=0.9427

🎯 Transformer training completed!
   Best performance: R² = 0.9526
   Training time: 186.26s
   Epochs trained: 9
   Model saved to: water_level_model_20250620-151437


In [ ]:
# --- BƯỚC 10: BIỂU ĐỒ ---
plt.figure(figsize=(12, 5))
plt.plot(y_test_inv, label='Thực tế')
plt.plot(y_pred_inv, label='Dự đoán')
plt.title('Dự báo trung bình mực nước trạm q64 (30 ngày tới) dùng dữ liệu từ tất cả trạm')
plt.xlabel('Mẫu')
plt.ylabel('Mực nước (gốc)')
plt.legend()
plt.grid(True)
plt.show()

/usr/local/lib/python3.11/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  fig.canvas.print_figure(bytes_io, **kw)


KeyboardInterrupt: 